In [1]:
# Solving the "uniform settling" problem in Lautrup p. 136-137
import numpy as np
import matplotlib.pyplot as plt
# from fenics import * 
import fenics

# Parameters
h   = 1   # Height (metre)
rho = 917  # kg/m^3
rho_w = 1029 # kg/m^3
g   = 9.8 # m/s^2

E = 2.5e9 # Pa
nu = 0.3

lam = (E*nu)/((1-2*nu)*(1+nu)) # lambda parameter

mu  = E/(2+2*nu)    # mu parameter

D   = (2*mu+lam)/(rho*g)

ModuleNotFoundError: No module named 'fenics'

In [ ]:
# The 1D mesh
N = 10 # Number of mesh nodes 
mesh = IntervalMesh(N, 0, h) # N_nodes on x=[0,h]
plt.figure();
plot(mesh); 

In [ ]:
# Function space
deg = 1 # Polynomial degree 
V   = FunctionSpace(mesh, 'Lagrange', deg); # Polynomial function space of order "deg" (=1 is linear, ...)
uz  = TrialFunction(V) # the unknown function
w   = TestFunction(V)  # the weight function 

# The FEM problem
a = -w.dx(0) * uz.dx(0) * dx # Syntax: d(u_z)/dx == u_z.dx(0)
L = w * 1/D * dx 

# Boundary conditions
# note that x coordiante is "x[0]" (first spatial dimension)
def bottom_boundary(x, on_boundary): return on_boundary and near(x[0], 0) 
bc1 = DirichletBC(V, Constant(0), bottom_boundary) # u_z = 0 (no displacement) at z=0 (bottom boundary)
bcs = [bc1]

# Compute solution
uz_sol = Function(V)       # Define a new function in the space "V" to stored the solution in.
solve(a == L, uz_sol, bcs); # Solve the linear FEM problem (a=L, i.e. Mx=b) for the unknown expansion coef. vector (x)

# Understanding uz_sol 
#print(uz_sol(0.55)) # you can get the solutoin at any point by interpolating in this way
#print(uz_sol.vector().get_local())

In [ ]:
# Plot solution
plt.figure();
plot(uz_sol, lw=4)

# Plot analytical solution
z = np.linspace(0, h, 10)
uz_ana = -(h**2-(h-z)**2)/(2*D)
plt.plot(z, uz_ana, '--', lw=4);
plt.xlabel('$z$ (m)');
plt.ylabel('$u_z(z)$ (m)');

In [ ]:
# Calculated potential energy (see p. 136)
sig_zz = (2*mu+lam)*uz_sol 
U_ela = assemble( 0.5*sig_zz*uz_sol * dx ) # Integral over the elastic energy density ("loaded spring" energy)
dU_pot = assemble( -rho*g*uz_sol * dx )    # Integral over the change in grav. potential energy density
print('U_ela =', U_ela)
print('dU_pot =', dU_pot)